# Cell 1: Verify GPU and System Information
Confirm that GPU is recognized and get CPU specifications

In [ ]:
# Cell 1: Verify GPU and System Information
import torch
import subprocess
import psutil

print("=" * 70)
print("SYSTEM INFORMATION")
print("=" * 70)

# GPU Information
print("\n=== GPU Information ===")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"\nGPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"GPU Capability: {torch.cuda.get_device_properties(0).major}.{torch.cuda.get_device_properties(0).minor}")

# Check nvidia-smi
print("\n=== NVIDIA-SMI Output ===")
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

# CPU Information
print("=== CPU Information ===")
physical_cores = psutil.cpu_count(logical=False)
logical_cores = psutil.cpu_count(logical=True)

print(f"CPU Cores (Physical): {physical_cores}")
print(f"CPU Threads (Logical): {logical_cores}")

cpu_model = subprocess.run(
    ["grep", "-m", "1", "model name", "/proc/cpuinfo"],
    capture_output=True,
    text=True
).stdout.strip().split(':')[1].strip()

print(f"CPU Model: {cpu_model}")

# Recommendation for NUM_WORKERS
recommended_workers = physical_cores
print(f"\n💡 Recommended NUM_WORKERS: {recommended_workers}")
print(f"   (Image preprocessing is I/O bound - you can experiment with {int(physical_cores * 1.5)}-{physical_cores * 2} for potentially better throughput)")

print("\n" + "=" * 70)

# Cell 2: Uninstall Incompatible Packages
Runpod GPU pods start with PyTorch pre-installed. This is likely to be an incompatible version. DeepSeek-OCR requires specific versions of vLLM and Transformers to work, so Runpod needs to be made ready to avoid dependency conflicts.

In [ ]:
# Cell 2: Uninstall Incompatible Packages
print("Uninstalling existing packages...")
!pip uninstall -y torch torchvision torchaudio vllm flash-attn

print("\n✓ Uninstallation complete")
print("⚠ Proceed to Cell 3 to install correct versions")

# Cell 3: Install All Dependencies
Install PyTorch, vLLM, transformers, and all other required packages. Also clone the DeepSeek-OCR repository.

- vLLM: 0.8.5
- Transformers: 4.57.1
- PyTorch: 2.6.0+cu118
- CUDA: 11.8

In [ ]:
# Cell 3: Install All Dependencies
import subprocess
import sys
import os

print("=" * 70)
print("INSTALLING ALL DEPENDENCIES")
print("=" * 70)

def check_package(package_name, version_check=None):
    """Check if package is installed and optionally verify version"""
    try:
        if package_name == "flash-attn":
            import flash_attn
            return True
        elif package_name == "fitz":
            import fitz
            return True
        else:
            module = __import__(package_name.replace('-', '_'))
            if version_check:
                return version_check(module)
            return True
    except:
        return False

# Install PyTorch 2.6.0 with CUDA 11.8
print("\n=== Installing PyTorch 2.6.0 ===")
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118

# Install vLLM 0.8.5
print("\n=== Installing vLLM 0.8.5 ===")
!pip install vllm==0.8.5 --no-cache-dir

# Install Flash Attention 2.7.3
print("\n=== Installing Flash Attention 2.7.3 ===")
!pip install flash-attn==2.7.3 --no-build-isolation

# Check and install Transformers (need >=4.51.1 for vLLM 0.8.5)
print("\n=== Checking Transformers ===")
try:
    import transformers
    version_parts = transformers.__version__.split('.')
    major, minor = int(version_parts[0]), int(version_parts[1])
    current_version = transformers.__version__
    
    if major > 4 or (major == 4 and minor >= 51):
        print(f"✓ Transformers {current_version} (compatible with vLLM 0.8.5)")
    else:
        print(f"⚠ Transformers {current_version} too old, installing >=4.51.1...")
        !pip install "transformers>=4.51.1" -q
except:
    print("Installing Transformers >=4.51.1...")
    !pip install "transformers>=4.51.1" -q

# Install additional dependencies
print("\n=== Installing Additional Dependencies ===")
packages_to_check = {
    'tokenizers': 'tokenizers==0.20.3',
    'fitz': 'PyMuPDF',
    'img2pdf': 'img2pdf',
    'einops': 'einops',
    'easydict': 'easydict',
    'addict': 'addict',
    'PIL': 'Pillow',
    'numpy': 'numpy',
    'hf_transfer': 'hf_transfer'
}

missing_packages = []
for module_name, pip_name in packages_to_check.items():
    if check_package(module_name):
        print(f"✓ {pip_name}")
    else:
        print(f"✗ {pip_name} - will install")
        missing_packages.append(pip_name)

if missing_packages:
    print(f"\nInstalling missing packages: {', '.join(missing_packages)}")
    !pip install {' '.join(missing_packages)} -q

# Clone DeepSeek-OCR Repository
print("\n=== Cloning DeepSeek-OCR Repository ===")
repo_path = '/workspace/DeepSeek-OCR-repo'

if os.path.exists(repo_path):
    print(f"✓ Repository already exists at {repo_path}")
else:
    print(f"Cloning repository to {repo_path}...")
    result = subprocess.run(
        ['git', 'clone', 'https://github.com/deepseek-ai/DeepSeek-OCR.git', repo_path],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("✓ Repository cloned successfully")
    else:
        print(f"✗ Failed to clone repository: {result.stderr}")

print("\n" + "=" * 70)
print("✓ All dependencies installed successfully!")
print("⚠ RESTART KERNEL before proceeding to Cell 4")
print("=" * 70)

# Cell 4: Setup and Verify Environment

In [ ]:
# Cell 4: Setup and Verify Environment
import torch
import os
import sys
from pathlib import Path

print("=" * 70)
print("ENVIRONMENT VERIFICATION")
print("=" * 70)

# Verify versions
print("\n=== Package Versions ===")
try:
    import vllm
    print(f"✓ vLLM: {vllm.__version__}")
    assert vllm.__version__ == "0.8.5", f"Wrong vLLM version: {vllm.__version__}"
except Exception as e:
    print(f"✗ vLLM error: {e}")

try:
    import transformers
    print(f"✓ Transformers: {transformers.__version__}")
except Exception as e:
    print(f"✗ Transformers error: {e}")

print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA: {torch.version.cuda}")

try:
    import flash_attn
    print("✓ Flash Attention: installed")
except:
    print("✗ Flash Attention: not found")

try:
    import fitz
    print("✓ PyMuPDF: installed")
except:
    print("✗ PyMuPDF: not found")

try:
    import einops
    print("✓ einops: installed")
except:
    print("✗ einops: not found")

# Verify GPU
print("\n=== GPU Status ===")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("✗ CUDA not available")

# Verify Repository
print("\n=== Repository Status ===")
repo_path = Path('/workspace/DeepSeek-OCR-repo')
vllm_impl_path = repo_path / 'DeepSeek-OCR-master' / 'DeepSeek-OCR-vllm'

if repo_path.exists():
    print(f"✓ Repository found at {repo_path}")
    if vllm_impl_path.exists():
        print(f"✓ vLLM implementation found at {vllm_impl_path}")
    else:
        print(f"⚠ vLLM implementation not found at expected path")
else:
    print(f"✗ Repository not found at {repo_path}")

# Setup workspace directories
print("\n=== Creating Workspace Directories ===")
workspace_dirs = [
    '/workspace/model',
    '/workspace/pdfs',
    '/workspace/temp',
    '/workspace/output',
    '/workspace/reports'
]

for dir_path in workspace_dirs:
    Path(dir_path).mkdir(parents=True, exist_ok=True)
    print(f"✓ {dir_path}")

print("\n" + "=" * 70)
print("✓ Environment ready!")
print("Proceed to Cell 5 to download the model")
print("=" * 70)

# Cell 5: Download DeepSeek-OCR Model

In [ ]:
# Cell 5: Download DeepSeek-OCR Model
from pathlib import Path
import os

print("=" * 60)
print("MODEL DOWNLOAD")
print("=" * 60)

# Configuration
model_name = "deepseek-ai/DeepSeek-OCR-master"
local_model_path = "/workspace/model/DeepSeek-OCR"
model_path = Path(local_model_path)

# Critical files that must exist for model to be valid
critical_files = [
    'config.json',
    'model.safetensors.index.json',
    'tokenizer.json',
    'tokenizer_config.json'
]

print(f"\nModel: {model_name}")
print(f"Destination: {local_model_path}")

should_download = False

if model_path.exists():
    print(f"\n⚠ Model directory exists, checking contents...")
    
    # Check if all critical files present
    missing_files = []
    for file in critical_files:
        if not (model_path / file).exists():
            missing_files.append(file)
    
    if missing_files:
        print(f"✗ Missing files: {', '.join(missing_files)}")
        print("Model appears incomplete, will re-download...")
        should_download = True
    else:
        print("✓ All critical files present")
        print("\nExisting model contents:")
        !ls -lh {local_model_path}
        print("\nModel size:")
        !du -sh {local_model_path}
        
        print("\n" + "="*60)
        response = input("Model exists. Re-download anyway? (y/N): ").strip().lower()
        should_download = response == 'y'
else:
    print("\n✓ Model directory does not exist, will download...")
    should_download = True

if should_download:
    print("\n" + "="*60)
    print("Downloading model (~7GB, may take several minutes)...")
    print("="*60)
    
    from huggingface_hub import snapshot_download
    
    try:
        snapshot_download(
            repo_id=model_name,
            local_dir=local_model_path,
            local_dir_use_symlinks=False
        )
        print(f"\n✓ Model downloaded successfully!")
    except Exception as e:
        print(f"\n✗ Download failed: {e}")
        print("You may need to authenticate with Hugging Face:")
        print("  Run: huggingface-cli login")
        raise
else:
    print("\n✓ Using existing model")

# Final verification
print("\n" + "="*60)
print("Model Verification")
print("="*60)

print("\nModel directory contents:")
!ls -lh {local_model_path}

print("\nTotal model size:")
!du -sh {local_model_path}

print("\nVerifying critical files:")
all_present = True
for file in critical_files:
    file_path = model_path / file
    if file_path.exists():
        print(f"  ✓ {file}")
    else:
        print(f"  ✗ {file} - MISSING!")
        all_present = False

if all_present:
    print("\n✓ Model ready to use!")
else:
    print("\n⚠ Model incomplete - some files missing")

print("="*60)
print("\n✓ Setup complete! You can now run the OCR workload notebook.")
print("="*60)